## Ocean carbon diagnostics for the SSP2-4.5 ensemble member 001

## CESM emissions-driven run tutorial

### By Kristen Krumhardt

# ---------------------------

### Import statements (using NPL 2023a environment)

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

import pop_tools 
import pandas as pd

### define case name and path

In [2]:
case = 'b.e21.BSSP245.f09_g17.hopens-reference.esmfirebvoc.001'
path = '/glade/campaign/cesm/collections/CCISEns/archive/' + case + '/ocn/hist/'

### Grab year 2015

In [3]:
files = []
year = 2015

yr4="{:04d}".format(year)

for month in range(1, 13):

    mo2="{:02d}".format(month)
    files.extend(sorted(glob(f'{path}/{case}.pop.h.{yr4}-{mo2}.nc')))


ds2015 = xr.open_mfdataset(files,decode_times=True,decode_coords=False, concat_dim='time', combine='nested')

### Grab year 2100

In [4]:
files = []
year = 2100

yr4="{:04d}".format(year)

for month in range(1, 13):

    mo2="{:02d}".format(month)
    files.extend(sorted(glob(f'{path}/{case}.pop.h.{yr4}-{mo2}.nc')))


ds2100 = xr.open_mfdataset(files,decode_times=True,decode_coords=False, concat_dim='time', combine='nested')

###  Specify desired variables and drop the rest

In [5]:
variables = ['coccoCaCO3','diatC', 'coccoC','spC','diazC',
             'microzooC','mesozooC','DIC','DOC','DOCr'] 

keep_vars = ['z_t','z_t_150m','time_bound','TAREA',
             'REGION_MASK','dz','KMT','TLONG','TLONG'] + variables

ds2015 = ds2015.drop([v for v in ds2015.variables if v not in keep_vars])
ds2100 = ds2100.drop([v for v in ds2100.variables if v not in keep_vars])

### Make an annual mean

In [6]:
ds2015 = ds2015.mean(dim='time')
ds2100 = ds2100.mean(dim='time')

### Use pop tools to get ocean grid info

In [7]:
ds_grid = pop_tools.get_grid('POP_gx1v7')
lats = ds_grid.TLAT
lons = ds_grid.TLONG
area = ds_grid.TAREA
area_m2 = area * 1.e-4
dz_m = ds_grid.dz * 0.01

In [8]:
area_m2.attrs['units'] = 'm2'
dz_m.attrs['units'] = 'm'

In [9]:
dz_m_150m = dz_m.isel(z_t=slice(0,15)).rename({'z_t':'z_t_150m'})

In [10]:
dz_m['z_t'] = ds2015.z_t
dz_m_150m['z_t'] = ds2015.z_t_150m

## Total up global carbon inventories

#### Divide concentration variables into two groups based on depth dimension

In [11]:
vars1 = ['coccoCaCO3','diatC', 'coccoC','spC','diazC','microzooC','mesozooC']
vars2 = ['DIC','DOC','DOCr']

#### sum up carbon into global datasets

In [12]:
ds_glb2015 = xr.Dataset()
ds_glb2100 = xr.Dataset()

for v in vars1:
    
    ### 2015
    tmp = (ds2015[v] * dz_m_150m).sum(dim='z_t_150m') # units = mmolC/m2
    tmp = tmp * area_m2 # units = mmol C/gridcell
    tmp = tmp.sum(dim=('nlon','nlat')) # mmol C in the ocean

    ds_glb2015[v] = tmp.values * 0.001 * 12.011 * 1e-15 # convert from mmol C to Pg C
    
    ### 2100
    tmp = (ds2100[v] * dz_m_150m).sum(dim='z_t_150m') # units = mmolC/m2
    tmp = tmp * area_m2 # units = mmol C/gridcell
    tmp = tmp.sum(dim=('nlon','nlat')) # mmol C in the ocean

    ds_glb2100[v] = tmp.values * 0.001 * 12.011 * 1e-15 # convert from mmol C to Pg C

for v in vars2:
    tmp = (ds2015[v] * dz_m).sum(dim='z_t') # units = mmolC/m2
    tmp = tmp * area_m2 # units = mmol C/gridcell
    tmp = tmp.sum(dim=('nlon','nlat')) # mmol C in the ocean

    ds_glb2015[v] = tmp.values * 0.001 * 12.011 * 1e-15 # convert from mmol C to Pg C
    
    
    ### 2100
    tmp = (ds2100[v] * dz_m).sum(dim='z_t') # units = mmolC/m2
    tmp = tmp * area_m2 # units = mmol C/gridcell
    tmp = tmp.sum(dim=('nlon','nlat')) # mmol C in the ocean

    ds_glb2100[v] = tmp.values * 0.001 * 12.011 * 1e-15 # convert from mmol C to Pg C
    

### Total up carbon in the ocean

In [13]:
totC_2015 = 0
totC_2100 = 0

for var in vars1 + vars2:

    totC_2015 = totC_2015 + ds_glb2015[var].values
    totC_2100 = totC_2100 + ds_glb2100[var].values

ds_glb2015['totC'] = totC_2015
ds_glb2100['totC'] = totC_2100

### Make a table of the values

In [15]:
df = pd.DataFrame(columns=['carbon pool','quantity 2015','quantity 2100','unit'])

In [16]:
df.loc[0] = ['Dissolved inorganic carbon',np.round(ds_glb2015.DIC.values,1),np.round(ds_glb2100.DIC.values,1),'Pg C']
df.loc[1] = ['Refractory dissolved organic carbon',np.round(ds_glb2015.DOCr.values,3),np.round(ds_glb2100.DOCr.values,3),'Pg C']
df.loc[2] = ['Semi-labile dissolved organic carbon',np.round(ds_glb2015.DOC.values,3),np.round(ds_glb2100.DOC.values,3),'Pg C']
df.loc[3] = ['Mesozooplankton biomass',np.round(ds_glb2015.mesozooC.values,3),np.round(ds_glb2100.mesozooC.values,3),'Pg C']
df.loc[4] = ['Microzooplankton biomass',np.round(ds_glb2015.microzooC.values,3),np.round(ds_glb2100.microzooC.values,3),'Pg C']
df.loc[5] = ['Diatom biomass',np.round(ds_glb2015.diatC.values,3),np.round(ds_glb2100.diatC.values,3),'Pg C']
df.loc[6] = ['Small phytoplankton biomass',np.round(ds_glb2015.spC.values,3),np.round(ds_glb2100.spC.values,3),'Pg C']
df.loc[7] = ['Coccolithophore biomass',np.round(ds_glb2015.coccoC.values,3),np.round(ds_glb2100.coccoC.values,3),'Pg C']
df.loc[8] = ['Diazotroph biomass',np.round(ds_glb2015.diazC.values,5),np.round(ds_glb2100.diazC.values,5),'Pg C']
df.loc[9] = ['Coccolithophore CaCO$_3$',np.round(ds_glb2015.coccoCaCO3.values,5),np.round(ds_glb2100.coccoCaCO3.values,5),'Pg C']
df.loc[10] = ['Total carbon in ocean',np.round(ds_glb2015.totC.values,2),np.round(ds_glb2100.totC.values,2),'Pg C']

In [17]:
df

,carbon pool,quantity 2015,quantity 2100,unit
0,Dissolved inorganic carbon,37233.60000,37479.30000,Pg C
1,Refractory dissolved organic carbon,361.57800,362.15700,Pg C
2,Semi-labile dissolved organic carbon,29.42600,30.10500,Pg C
3,Mesozooplankton biomass,0.41400,0.41200,Pg C
4,Microzooplankton biomass,0.23300,0.23300,Pg C
5,Diatom biomass,0.26100,0.25600,Pg C
6,Small phytoplankton biomass,0.19600,0.19600,Pg C
7,Coccolithophore biomass,0.03700,0.03500,Pg C
8,Diazotroph biomass,0.00986,0.00997,Pg C
9,Coccolithophore CaCO$_3$,0.01398,0.01217,Pg C
